In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN', 'NAME'], axis=1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [36]:
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
application_types_to_replace = application_type_counts[application_type_counts < 200].index.tolist()

In [37]:
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app, 'Other')

In [38]:
classification_counts = application_df['CLASSIFICATION'].value_counts()
classifications_to_replace = classification_counts[classification_counts < 1000].index.tolist()

In [39]:
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls, 'Other')


In [40]:
application_df_encoded = pd.get_dummies(application_df, columns=['APPLICATION_TYPE', 'CLASSIFICATION'])

In [41]:
X = application_df_encoded.drop('IS_SUCCESSFUL', axis=1)
y = application_df_encoded['IS_SUCCESSFUL']

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [44]:
numeric_features = X.select_dtypes(include=['float64', 'int64']).columns
non_numeric_features = X.select_dtypes(include=['object']).columns

In [45]:
scaler = StandardScaler()
X_train_scaled_numeric = scaler.fit_transform(X_train[numeric_features])
X_test_scaled_numeric = scaler.transform(X_test[numeric_features])

In [47]:
X_train_scaled_array = X_train_scaled.values.astype(np.float32)
y_train_array = y_train.astype(np.float32).values

In [52]:
# Build a Sequential model
optimized_model = tf.keras.models.Sequential()

# Input layer
optimized_model.add(tf.keras.layers.Dense(units=128, activation='relu', input_dim=X_train_scaled.shape[1]))

# Hidden layers
optimized_model.add(tf.keras.layers.Dropout(0.5))
optimized_model.add(tf.keras.layers.Dense(units=64, activation='relu'))
optimized_model.add(tf.keras.layers.Dense(units=32, activation='relu'))

# Output layer
optimized_model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Compile the model
optimized_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Display the model summary
optimized_model.summary()

# Train the model with early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = optimized_model.fit(X_train_scaled_array, y_train_array, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_36 (Dense)            (None, 128)               1024      
                                                                 
 dropout_11 (Dropout)        (None, 128)               0         
                                                                 
 dense_37 (Dense)            (None, 64)                8256      
                                                                 
 dense_38 (Dense)            (None, 32)                2080      
                                                                 
 dense_39 (Dense)            (None, 1)                 33        
                                                                 
Total params: 11393 (44.50 KB)
Trainable params: 11393 (44.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).

In [13]:
X_train_encoded = pd.get_dummies(X_train, columns=['AFFILIATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS'])

In [15]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train_encoded)

# Scale the data
X_train_scaled = X_scaler.transform(X_train_encoded)
X_test_encoded = pd.get_dummies(X_test, columns=['AFFILIATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS'])
X_test_encoded = X_test_encoded.reindex(columns=X_train_encoded.columns, fill_value=0)
X_test_scaled = X_scaler.transform(X_test_encoded)

In [28]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [29]:
optimized_model = Sequential()

In [35]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

optimized_model = Sequential()
optimized_model.add(Dense(units=128, activation='relu', input_dim=X_train_encoded.shape[1]))
optimized_model.add(Dropout(0.5))
optimized_model.add(Dense(units=64, activation='relu'))
optimized_model.add(Dense(units=32, activation='relu'))
optimized_model.add(Dense(units=1, activation='sigmoid'))

optimized_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

history = optimized_model.fit(X_train_encoded, y_train, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/50
686/686 [==============================] - 2s 2ms/step - loss: 26457.9570 - accuracy: 0.4930 - val_loss: 407800.7812 - val_accuracy: 0.5171
Epoch 2/50
686/686 [==============================] - 1s 2ms/step - loss: 86796.1250 - accuracy: 0.4934 - val_loss: 16161.5820 - val_accuracy: 0.4829
Epoch 3/50
686/686 [==============================] - 1s 2ms/step - loss: 37563.1758 - accuracy: 0.4844 - val_loss: 169981.5000 - val_accuracy: 0.5171
Epoch 4/50
686/686 [==============================] - 1s 2ms/step - loss: 24326.0605 - accuracy: 0.5033 - val_loss: 233.7271 - val_accuracy: 0.6317
Epoch 5/50
686/686 [==============================] - 1s 2ms/step - loss: 27966.2148 - accuracy: 0.4900 - val_loss: 70878.9141 - val_accuracy: 0.5171
Epoch 6/50
686/686 [==============================] - 1s 2ms/step - loss: 16169.0088 - accuracy: 0.4942 - val_loss: 241.7447 - val_accuracy: 0.4829
Epoch 7/50
686/686 [==============================] - 1s 2ms/step - loss: 12703.2188 - accuracy: 0.496